# Multiplexing Exercise - Monte Carlo π

A simple toy problem to get a handle on multiple engines is a Monte
Carlo approximation of π.

Let's say we have a dartboard with a round target inscribed on a square
board. If you threw darts randomly, and they land evenly distributed on
the square board, how many darts would you expect to hit the target?

<img src="../figs/darts.png"/>

$$
\frac{A_c}{A_{sq}} = \frac{\pi r^2}{(2r)^2} = \frac{\pi}{4}
$$

In [1]:
from __future__ import print_function

from random import random
from math import pi

def mcpi(nsamples):
    s = 0
    for i in range(nsamples):
        x = random()
        y = random()
        if x*x + y*y <= 1:
            s+=1
    return 4.*s/nsamples

In [2]:
for n in [10, 100, 1000, 10000, 100000, 1000000]:
    print("%8i" % n, end=' ')
    for i in range(3):
        print("%.5f" % mcpi(n), end=' ')
    print()

      10 2.80000 2.40000 3.60000 
     100 3.08000 3.00000 2.84000 
    1000 3.10000 3.16800 3.10800 
   10000 3.13000 3.15280 3.13880 
  100000 3.14468 3.14900 3.13928 
 1000000 3.13997 3.14320 3.14118 


In [3]:
%timeit mcpi(1000000)

254 ms ± 5.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It takes a lot of samples to get a good approximation. Can you write a
function that will use your engines to break up the work?

```python
def multi_mcpi(dview, nsamples):
    raise NotImplementedError("you write this")
```

In [4]:
import ipyparallel as ipp
rc = ipp.Client()

view = rc[:]

In [ ]:
def multi_mcpi(dview, nsamples):
    raise NotImplementedError("you write this")

In [9]:
# %load ../soln/mcpi.py
def subsample(nsamples):
    from random import random
    s = 0
    for i in range(nsamples):
        x = random()
        y = random()
        if x*x + y*y <= 1:
            s += 1
    return s

def multi_mcpi(view, nsamples):
    p = len(view.targets)
    if nsamples % p:
        # ensure even divisibility
        nsamples += p - (nsamples % p)
    
    subsamples = nsamples // p
    
    ar = view.apply_async(subsample, subsamples)
    return 4. * sum(ar) / nsamples

In [11]:
multi_mcpi(view, 10000000)

3.1413148

In [12]:
N = int(1e7)
%time mcpi(N)

CPU times: user 2.65 s, sys: 17.2 ms, total: 2.67 s
Wall time: 2.68 s


3.141932

In [14]:
%time multi_mcpi(view, N)

CPU times: user 13.9 ms, sys: 2.53 ms, total: 16.5 ms
Wall time: 1.28 s


3.1413616

In [ ]:
# bonus: plot convergence like we did for Monte Carlo integration